# CV RAG Training Notebook (Deep Dive - Hybrid Search)

Tento notebook jde "pod kapotu". Nebudeme jen slepě spouštět metody, ale podíváme se dovnitř objektů.
Cílem je pochopit:
1.  Co přesně je v BM25 indexu.
2.  Co přesně je ve Vektorovém indexu.
3.  Jak se tyto dva světy potkávají.

**DŮLEŽITÉ:** Zde budeme sahat na "privátní" atributy (začínající podtržítkem), což se v produkci nedělá, ale pro výuku je to nutné.

In [1]:
import sys
import logging
from pathlib import Path

sys.path.insert(0, str(Path.cwd().parent))

# Nastavíme logging na INFO, abychom viděli co se děje
logging.basicConfig(
    level=logging.INFO,
    format='%(name)s - %(message)s'
)

from src.config import AppConfig
from src.document_loader import CVDocumentLoader
from src.embeddings import EmbeddingsManager
from src.vector_store import VectorStoreManager
from src.parent_retriever import CVParentRetriever

## 1. Setup (Rychlý průchod)
Stejné jako minule, připravíme data a retriever se zapnutým Hybrid Search.

In [2]:
config = AppConfig()
config.rag.use_hybrid_search = True  # VYNUCENÍ HYBRID SEARCH

# Načtení dat
loader = CVDocumentLoader(config.rag.data_directory_ntb)
candidates = loader.load_all_cvs()
documents = loader.convert_to_langchain_documents(candidates)

# Setup Store
embeddings_mgr = EmbeddingsManager(config.azure)
vs_manager = VectorStoreManager(config.rag, embeddings_mgr.get_embeddings())
vs_manager.clear_vectorstore()
vectorstore = vs_manager.create_or_load_vectorstore()

# Inicializace Retrieveru
retriever = CVParentRetriever(
    config=config.rag,
    vectorstore=vectorstore,
    azure_config=config.azure
)
retriever.initialize_retriever(documents)

print("\n✅ Retriever inicializován.")

src.document_loader - Found 27 DOCX files in ..\data\OneDrive_2025-12-16
src.document_loader - Loaded CV for Baláček Daniel (3020 characters)
src.document_loader - Loaded CV for Bobůrka Vojtěch (2458 characters)
src.document_loader - Loaded CV for Bronec Ondřej (3757 characters)
src.document_loader - Loaded CV for Bukovský Petr (2628 characters)
src.document_loader - Loaded CV for Bímová Kamila (2042 characters)
src.document_loader - Loaded CV for Dlugošová Lenka (2383 characters)
src.document_loader - Loaded CV for Duleba Peter (2873 characters)
src.document_loader - Loaded CV for Fejfarová Julia (1445 characters)
src.document_loader - Loaded CV for Fejfar Ondřej (1289 characters)
src.document_loader - Loaded CV for Gleb Tcypin (2543 characters)
src.document_loader - Loaded CV for Hlavatá Michaela (5445 characters)
src.document_loader - Loaded CV for Hlinková Zuzana (2615 characters)
src.document_loader - Loaded CV for Holman Martin (1559 characters)
src.document_loader - Loaded CV fo


✅ Retriever inicializován.


## 2. DEEP DIVE: Co je uvnitř?

Nyní se podíváme přímo do střev objektu `retriever`. Zajímají nás dvě hlavní komponenty:
1.  `bm25_retriever` - Hledání klíčových slov
2.  `embedding_retriever` - Vektorové hledání

In [3]:
# Získáme instanci HybridRetrieveru (která je schovaná v _hybrid_retriever)
hybrid_core = retriever._hybrid_retriever

print(f"Hybrid Core Object: {hybrid_core}")
print(f"BM25 Retriever: {hybrid_core.bm25_retriever}")
print(f"Embedding Retriever: {hybrid_core.embedding_retriever}")

Hybrid Core Object: <src.hybrid_retriever.HybridRetriever object at 0x000002F920CCE660>
BM25 Retriever: vectorizer=<rank_bm25.BM25Okapi object at 0x000002F920CCEBA0> k=10
Embedding Retriever: tags=['Chroma', 'AzureOpenAIEmbeddings'] vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000002F91E2474D0> search_kwargs={'k': 10}


### A. Analýza BM25 (Klíčová slova)
Co BM25 indexuje? Celé dokumenty (Parent Chunks) nebo malé kousky (Child Chunks)?
Zkusíme vyhledat slovo a podíváme se na délku výsledku.

In [5]:
query = "python"

# Zavoláme PŘÍMO BM25 část (izolovaně)
bm25_results = hybrid_core.bm25_retriever.invoke(query)

print(f"🔍 BM25 hledání slova '{query}':")
print(f"   Počet výsledků: {len(bm25_results)}")

if bm25_results:
    doc = bm25_results[0]
    print(f"\n   První výsledek:")
    print(f"   Autor: {doc.metadata.get('candidate_name')}")
    print(f"   Délka textu: {len(doc.page_content)} znaků (Dlouhý text = Parent Chunk)")
    print(f"   Ukázka: {doc.page_content[:]}...")
else:
    print("   Žádné výsledky pro BM25.")

🔍 BM25 hledání slova 'python':
   Počet výsledků: 10

   První výsledek:
   Autor: Konvalinka Michal
   Délka textu: 1997 znaků (Dlouhý text = Parent Chunk)
   Ukázka: www.dolphinconsulting.cz	

Michal Konvalinka

Instructor and Consultant of BI and MS Office Applications 

 Prague



Key qualifications

Power BI instructor and consultant

Data cleansing, transformation, analysis, and visualization

Microsoft products instructor (Excel, Microsoft 365)

Learning content designer and developer

Secondary and tertiary education educator since 2014



Skills & knowledge

Business intelligence

Microsoft Power BI

MS Excel, Power Query, Power Pivot

Database applications

SQL, data modeling, MS Access

Microsoft office suite 

Expert knowledge and skills in MS Word, PowerPoint, Outlook, and Microsoft 365 – experience in training at all levels since 2009

Learning design

Design and implementation of educational content for various public and private schools (Business Academy, Prague Univers

### B. Analýza Vektorů (Sémantika)
Co vrátí vektorové hledání? Stejný dokument? Nebo menší část?

In [ ]:
# Zavoláme PŘÍMO Vektorovou část (izolovaně)
vec_results = hybrid_core.embedding_retriever.invoke(query)

print(f"🔍 Vektorové hledání slova '{query}':")
print(f"   Počet výsledků: {len(vec_results)}")

if vec_results:
    doc = vec_results[0]
    print(f"\n   První výsledek:")
    print(f"   Autor: {doc.metadata.get('candidate_name')}")
    print(f"   Délka textu: {len(doc.page_content)} znaků (Krátký text = Child Chunk)")
    print(f"   Ukázka: {doc.page_content[:150]}...")
else:
    print("   Žádné výsledky pro vektory.")

### C. Závěr z pitvy
Vidíte ten rozdíl?
- **BM25** vrací **Parent Documents** (celá CV).
- **Vector Store** vrací **Child Chunks** (odstavce).

To je důležité vědět! Když Hybrid Search dělá "fúzi", snaží se spojit tyto dva seznamy. Pokud by se algoritmus spoléhal na to, že texty jsou identické, fúze by selhala. Zde se RRF (Reciprocal Rank Fusion) děje na základě klíče vytvořeného z `candidate_name` + začátek `page_content`. 

Protože text je jiný (dlouhý vs krátký), pravděpodobně nedochází k "overlapu" (posílení skóre), ale spíše k "sjednocení" (union) výsledků. V `manual_query.ipynb` se podíváme, jestli to tak opravdu je.